In [5]:
from music21 import *
import keras
from keras import backend as k
import glob
import numpy
from keras.utils import np_utils 
from keras.models import Sequential
from keras.layers import Dense, Activation,Dropout,LSTM
from keras.optimizers import RMSprop
import librosa

In [8]:
notes = []
counter = 0
for file in glob.glob("midi_songs/*.mid"):
    print('processing song no: ',counter)
    counter = counter + 1
    midi = converter.parse(file)
    notes_to_parse = None
    parts = instrument.partitionByInstrument(midi)
    if parts: # file has instrument parts
        notes_to_parse = parts.parts[0].recurse()
    else: # file has notes in a flat structure
        notes_to_parse = midi.flat.notes
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))

processing song no:  0
processing song no:  1
processing song no:  2
processing song no:  3
processing song no:  4
processing song no:  5
processing song no:  6
processing song no:  7
processing song no:  8
processing song no:  9
processing song no:  10
processing song no:  11
processing song no:  12
processing song no:  13
processing song no:  14
processing song no:  15
processing song no:  16
processing song no:  17
processing song no:  18
processing song no:  19
processing song no:  20
processing song no:  21
processing song no:  22
processing song no:  23
processing song no:  24
processing song no:  25
processing song no:  26
processing song no:  27
processing song no:  28
processing song no:  29
processing song no:  30
processing song no:  31
processing song no:  32
processing song no:  33
processing song no:  34
processing song no:  35
processing song no:  36
processing song no:  37
processing song no:  38
processing song no:  39
processing song no:  40
processing song no:  41
pr

In [69]:
sequence_length = 100
n_vocab = len(set(notes))

# get all pitch names
pitchnames = sorted(set(item for item in notes))
# create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
network_input = []
network_output = []
# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])
n_patterns = len(network_input)
# reshape the input into a format compatible with LSTM layers
network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
# normalize input
network_input = network_input / float(n_vocab) 
network_output = np_utils.to_categorical(network_output)

In [70]:
model = Sequential()
model.add(LSTM(
        256,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [77]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_13 (LSTM)               (None, 100, 512)          1052672   
_________________________________________________________________
dropout_13 (Dropout)         (None, 100, 512)          0         
_________________________________________________________________
lstm_14 (LSTM)               (None, 100, 512)          2099200   
_________________________________________________________________
dropout_14 (Dropout)         (None, 100, 512)          0         
_________________________________________________________________
lstm_15 (LSTM)               (None, 512)               2099200   
_________________________________________________________________
dense_8 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_15 (Dropout)         (None, 256)               0         
__________

In [72]:
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"    
checkpoint = keras.callbacks.ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=0,        
    save_best_only=True,        
    mode='min'
)    
callbacks_list = [checkpoint]     
model.fit(network_input, network_output, epochs=200, batch_size=512, callbacks=callbacks_list)

Epoch 1/200


KeyboardInterrupt: 

In [80]:
# now we will generate the music
model = Sequential()
model.add(LSTM(
    512,
    input_shape=(network_input.shape[1], network_input.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
# Load the weights to each node
model.load_weights('weights.hdf5')

In [121]:
start = numpy.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
pattern = network_input[start]

prediction_output = []
# generate 500 notes
for note_index in range(500):
    prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)
    prediction = model.predict(prediction_input, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)
    ind =numpy.asarray([index])
    pattern = numpy.append(pattern, ind)
    pattern = pattern[1:len(pattern)]

In [122]:
offset = 0
output_notes = []
# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    # increase offset each iteration so that notes do not stack
    offset += 0.5

In [125]:
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='test_output.mid')

'test_output.mid'

In [8]:
# playing the file
mf = midi.MidiFile()
mf.open('test_output.mid')
mf.read()
mf.close()
s = midi.translate.midiFileToStream(mf)
s.show('midi')